In [1]:
%matplotlib inline


[Learn the Basics](https://pytorch.org/tutorials/beginner/basics/intro.html) ||
[**Quickstart**](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html) || 
[Tensors](https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html) || 
[Datasets & DataLoaders](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) ||
[Transforms](https://pytorch.org/tutorials/beginner/basics/transforms_tutorial.html) ||
[Build Model](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html) ||
[Autograd](https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html) ||
[Optimization](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html) ||
[Save & Load Model](https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html)

Quickstart
===================
This section runs through the API for common tasks in machine learning. Refer to the links in each section to dive deeper.

Working with data
-----------------
PyTorch has two [primitives to work with data](https://pytorch.org/docs/stable/data.html): 
``torch.utils.data.DataLoader`` and ``torch.utils.data.Dataset``.
``Dataset`` stores the samples and their corresponding labels, and ``DataLoader`` wraps an iterable around
the ``Dataset``.




In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

PyTorch offers domain-specific libraries such as [TorchText](https://pytorch.org/text/stable/index.html), 
[TorchVision](https://pytorch.org/vision/stable/index.html), and [TorchAudio](https://pytorch.org/audio/stable/index.html), 
all of which include datasets. For this tutorial, we  will be using a TorchVision dataset.

The ``torchvision.datasets`` module contains ``Dataset`` objects for many real-world vision data like 
CIFAR, COCO ([full list here](https://pytorch.org/docs/stable/torchvision/datasets.html)). In this tutorial, we
use the FashionMNIST dataset. Every TorchVision ``Dataset`` includes two arguments: ``transform`` and
``target_transform`` to modify the samples and labels respectively.



In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

We pass the ``Dataset`` as an argument to ``DataLoader``. This wraps an iterable over our dataset, and supports
automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element 
in the dataloader iterable will return a batch of 64 features and labels.



In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

print("Length of train_dataloader        : ", len(train_dataloader))
print("Length of train_dataloader.dataset: ", len(train_dataloader.dataset))
print("")
for batch, (X, y) in enumerate(train_dataloader):
    if batch == 0:
        print("Shape of first X [N, C, H, W]: ", X.shape)
        print("Shape of first y: ", y.shape, y.dtype)
        print("")
    if batch == len(train_dataloader) - 1:
        print("Shape of last X [N, C, H, W]: ", X.shape)
        print("Shape of last y: ", y.shape, y.dtype)
        print("")

print("= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =")

print("Length of test_dataloader         : ", len(test_dataloader))
print("Length of test_dataloader.dataset : ", len(test_dataloader.dataset))
print("")
for batch, (X, y) in enumerate(test_dataloader):
    if batch == 0:
        print("Shape of first X [N, C, H, W]: ", X.shape)
        print("Shape of first y: ", y.shape, y.dtype)
        print("")
    if batch == len(test_dataloader) - 1:
        print("Shape of last X [N, C, H, W]: ", X.shape)
        print("Shape of last y: ", y.shape, y.dtype)
        print("")

Length of train_dataloader        :  938
Length of train_dataloader.dataset:  60000

Shape of first X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of first y:  torch.Size([64]) torch.int64

Shape of last X [N, C, H, W]:  torch.Size([32, 1, 28, 28])
Shape of last y:  torch.Size([32]) torch.int64

= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
Length of test_dataloader         :  157
Length of test_dataloader.dataset :  10000

Shape of first X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of first y:  torch.Size([64]) torch.int64

Shape of last X [N, C, H, W]:  torch.Size([16, 1, 28, 28])
Shape of last y:  torch.Size([16]) torch.int64



Read more about [loading data in PyTorch](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).




--------------




Creating Models
------------------
To define a neural network in PyTorch, we create a class that inherits 
from [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). We define the layers of the network
in the ``__init__`` function and specify how data will pass through the network in the ``forward`` function. To accelerate 
operations in the neural network, we move it to the GPU if available.



In [5]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


Read more about [building neural networks in PyTorch](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html).




--------------




Optimizing the Model Parameters
----------------------------------------
To train a model, we need a [loss function](https://pytorch.org/docs/stable/nn.html#loss-functions)
and an [optimizer](https://pytorch.org/docs/stable/optim.html). 



In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and 
backpropagates the prediction error to adjust the model's parameters. 



In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model's performance against the test dataset to ensure it is learning.



In [8]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            batch_size = X.shape[0]
            # Why does official documentation not multiply batch_size?
            test_loss += loss_fn(pred, y).item() * batch_size
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (*epochs*). During each epoch, the model learns 
parameters to make better predictions. We print the model's accuracy and loss at each epoch; we'd like to see the
accuracy increase and the loss decrease with every epoch.



In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.310168  [    0/60000]
loss: 2.299865  [ 6400/60000]
loss: 2.292969  [12800/60000]
loss: 2.283962  [19200/60000]
loss: 2.275664  [25600/60000]
loss: 2.284712  [32000/60000]
loss: 2.276313  [38400/60000]
loss: 2.270878  [44800/60000]
loss: 2.284147  [51200/60000]
loss: 2.259291  [57600/60000]
Test Error: 
 Accuracy: 29.4%, Avg loss: 2.254207 

Epoch 2
-------------------------------
loss: 2.279468  [    0/60000]
loss: 2.255437  [ 6400/60000]
loss: 2.234116  [12800/60000]
loss: 2.215303  [19200/60000]
loss: 2.193687  [25600/60000]
loss: 2.217208  [32000/60000]
loss: 2.201185  [38400/60000]
loss: 2.196049  [44800/60000]
loss: 2.224796  [51200/60000]
loss: 2.127778  [57600/60000]
Test Error: 
 Accuracy: 42.1%, Avg loss: 2.154845 

Epoch 3
-------------------------------
loss: 2.224870  [    0/60000]
loss: 2.175960  [ 6400/60000]
loss: 2.130129  [12800/60000]
loss: 2.089783  [19200/60000]
loss: 2.053715  [25600/60000]
loss: 2.100030  [32000/600

Read more about `Training your model <optimization_tutorial.html>`_.




--------------




Saving Models
-------------
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).



In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Loading Models
----------------------------

The process for loading a model includes re-creating the model structure and loading
the state dictionary into it. 



In [11]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

This model can now be used to make predictions.



In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


Read more about [Saving & Loading your model](https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html).


